# Problem 1

In [122]:
%matplotlib inline
from __future__ import division
import scipy
from scipy.stats import norm
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.cm as cm


from math import sqrt, pi, exp, log

plt.rcParams['figure.figsize'] = (10.0, 8.0)
A = np.array([[0.990, 0.005, 0.005], [0.005, 0.990, 0.005], [0.005, 0.005, 0.990]])
pi0 = [1/3, 1/3, 1/3]
Q = [-1, 0, 1]
T = 500
scale=0.0001
colors = cm.rainbow(np.linspace(0, 1, 6))
scipy.set_printoptions(precision = 4, suppress = True)


In [123]:
def Yt(x):
    return norm.rvs(loc=x, scale=scale, size=1)[0]

def run_hmm():
    starting_state = np.random.choice(Q,p=pi0)
    prev_state = starting_state
    xt = [prev_state]
    yt = [Yt(prev_state)]
    #print(xt,yt)
    for t in range(1,T):
        next_state = np.random.choice(Q, p=A[xt[t-1]+1,:])
        xt.append(next_state)
        yt.append(Yt(next_state)) 
    return {'xt': xt, 'yt': yt}

In [127]:
np.random.seed(100)

def b(x,m):
    p1 = norm.pdf(x, loc=m, scale=scale)
    #if p>1:
    #    raise RuntimeError('error',p)
    p = -0.5* log(2*pi*scale) + -(x-m)**2/(2*scale)
    #print(p,x,m)
    return p

def viterbi(yt):
    delta = [{} for t in range(0,T)]
    path = {}
    for q in Q:
        delta[0][q] = log(pi0[q+1])+log(b(yt[0],q)) 
        path[q] = [q]
    #print path
    for t in range(1,T):
        tempath = {}
        for q in Q:
            (Z, state) =  max(( delta[t-1][x] + b(yt[t],q) + log(A[x+1,q+1]),x) for x in Q)
            delta[t][q] = Z
            #print(q,state)
            tempath[q] = path[state]+[q]
        path = tempath
        #print path
    #print(delta)
    (p, state) = max((delta[T-1][q], q) for q in Q)
    #print(p)
    return path[state],delta

for i in range(0,1):
    d = run_hmm()
    #ax = fig.add_subplot(3,2,i)
    xt = d['xt']
    yt = d['yt']
    #plt.scatter(xt, yt, color=colors[i])
    xtp, delta = viterbi(yt)
    #print(xt)
    #print(xtp)
    plt.scatter(xtp, xt, color=colors[i])

ValueError: math domain error

# Problem 2

## The above problem of course has a grave error. After Dinner.

In [68]:
1.5*1.5

2.25

In [69]:
(2-2.5)*(1-2.5) + (4-2.5)*(2-2.5) + (-2-2.5)*(3-2.5) + (6-2.5)*(4-2.5)

3.0

In [71]:
(2-2.5)**2 + (4-2.5)**2 + (-2-2.5)**2 + (6-2.5)**2

35.0

In [72]:
2.5*0.4

1.0

In [73]:
0.4**2+1.8**2+4.8**2+2.6**2

33.199999999999996

In [74]:
1-33.19/35.0

0.051714285714285824

In [75]:
vif1 = 35/33.19

In [76]:
vif1

1.0545344983428744

In [71]:
max((i,x) for i,x in enumerate(Q))

(2, 1)

In [128]:
for q in Q:
    print(pi)
    delta[0][q] = log(pi0[q+1])+log(b(yt[0],q)) 

ValueError: math domain error